In [84]:
import pandas as pd
import numpy as np
import string
import openpyxl
import random
import matplotlib.pyplot as plt

# 주문 내역 생성

* 칼럼별 independent하게 random choice 하는 모델

* unique 특성 --> dictionary
* 특성 랜텀 선택함수
* 주문 내역 생성함수

In [85]:
columns=['플랜트', '플랜트명', '차수', '주문일자', '매장코드', '매장', '상품코드', '상품명', '운송장번호',
       '운송장출력여부', '주문취소여부', '주문번호', '받는이', '우편번호', '주소1', '주소2', '전화번호1',
       '전화번호2', '배송기재사항', '판매가', '합포여부', '예정수량', '검수수량', '출력일시', '출력자', '몰구분',
       'DAS여부', 'DAS차수', 'DAS지시번호', 'BOXNO', 'CELLNO', '추가박스여부']

In [86]:
#샘플데이터 받아서 딕셔너리 자료형으로 변환 특성 종류 : 특성 value
Sample_set = {}
for col in columns:
    f = open('C:/geunskoo-github-blog/ffm-logistics-planning/CODE/Sample/' + col,'r',encoding ='utf-8')     #안호성이 만든 CODE/Sample/         각자의 저장 주소 참고하세요!
    data = f.read()[:-1].split('|')
    Sample_set[col] = data

#input - 특성 종류
#output - 랜덤 특성 value
def one_column_random(name):
    return random.choice(Sample_set[name])

#input - 주문 수, "excel파일 이름"
#output - excel 파일 생성
def random_order_creater(n, name = 'Random_Sample'):
    result = pd.DataFrame([],columns=columns)
    for _ in range(n):
        new_random_data = []
        for col in columns:
            new_random_data.append(one_column_random(col))
        result.loc[len(result)] = new_random_data
    result.to_excel(name+'.xlsx',encoding='utf-8')
    print('파일이 생성되었습니다.')

#실행
random_order_creater(1000)

파일이 생성되었습니다.


# 개선점 1

* 특성들이 너무 독립적임 --> 개별 특성이 갖는 성질들을 고려해주어야함.


* 플랜트 ---> 플랜트 명
* 매장코드 ---> 매장
* 상품코드 ---> 상품명

## 상관관계 파일 만들기

In [88]:
data = pd.read_excel("C:/Users/oryuk/OneDrive/바탕 화면/202204_출고_주문번호추가.xlsx", skiprows=[0]) # <---- 각자 적재데이터파일 오픈하면 됩니다. 
data.head()

,No,플랜트,플랜트명,차수,주문일자,매장코드,매장,상품코드,상품명,운송장번호,...,검수수량,출력일시,출력자,몰구분,DAS여부,DAS차수,DAS지시번호,BOXNO,CELLNO,추가박스여부
0,1.0,192,프리덤(0192),1.0,2022/04/21,0192-1,프리덤 택배출고,802980111116,히말라야 핑크솔트,5.620936e+11,...,1,NaN,NaN,NaN,N,0.0,NaN,NaN,NaN,N
1,2.0,120,제로투원(0120),1.0,2022/04/01,0120-1,제로투원 택배출고,6261108117326,샤프란 향신료 20g,5.620932e+11,...,1,NaN,NaN,NaN,N,0.0,NaN,NaN,NaN,N
2,3.0,176,창고이전-더식스코리아(0176),1.0,2022/04/01,0176-01,더식스코리아-택배,5285001951833,Elegance Plus Shaving Gel 500ml,5.620932e+11,...,1,NaN,NaN,NaN,N,0.0,NaN,NaN,NaN,N
3,4.0,106,올댓글로벌(0106),1.0,2022/04/06,0106-1,올댓글로벌 택배,8000070021341,원두커피: 3. 라바짜 그란에스프레소,5.620933e+11,...,6,NaN,NaN,NaN,N,0.0,NaN,NaN,NaN,N
4,5.0,203,(주)디앤씨웹툰비즈(0203),1.0,2022/04/06,0203,디앤씨웹툰비즈-택배,VD-MT-002,악역의 엔딩은 죽음뿐 마스킹 테이프 vol.2,5.620933e+11,...,4,NaN,NaN,NaN,N,0.0,NaN,NaN,NaN,N


* 상품명에 이상치 제거 필요

In [89]:
data['상품명'] = data['상품명'].replace('\n','',regex=True)
data['상품코드'] = data['상품코드'].replace('\n','',regex=True)

## 상관관계 making 함수 .ver1

In [90]:
def make_relation():
    one_to_one = []
    relation_list =[('플랜트','플랜트명'),('매장코드','매장'),('상품코드','상품명')]
    
    for key,value in relation_list:
        relation = []
        for i in range(len(data[key])):
            if (data[key][i],data[value][i]) not in relation:
                relation.append((data[key][i],data[value][i]))
        one_to_one.append(relation)       
    return one_to_one

## 상관관계 making 함수 .ver2

***플랜트 --->플랜트명, 매장코드, 매장 4개의 columns가 다 상관관계를 가진다.***

**.ver1의 문제점 : 플랜트 ---> 플랜트명, 매장코드---> 매장 으로 진행하면 어색한 상황이 생김.**
>ex) 플랜트: '001', 플랜트명: '롯데(001)' but 매장코드: '002-1', 매장: '신세계 택배'

In [91]:
dependent_list = [['플랜트','플랜트명','매장코드','매장'],['상품코드','상품명']]
#Input : 상관관계 2차원 list
#Output : 상관관계 txt파일 생성
def make_relation(relation_list):
    result = []
    for relation in relation_list:
        relation_value = []
        for i in range(len(data[relation[0]])):
            one_to_one = []
            for col in relation:
                one_to_one.append(data[col][i])
            if one_to_one not in relation_value:
                relation_value.append(one_to_one)
        result.append(relation_value)
    
    result = np.array(result)
    for i in range(len(result)):
        np.savetxt('relation '+ str(i+1) + '.txt',result[i], fmt='%s',delimiter='|')
        print(f'파일생성----> relation {i+1}.txt 파일이 생성되었습니다.')
        
#실행
make_relation(dependent_list)

파일생성----> relation 1.txt 파일이 생성되었습니다.
파일생성----> relation 2.txt 파일이 생성되었습니다.


C:\Users\oryuk\AppData\Local\Temp\ipykernel_17960\2498707524.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result)


## `상관관계 함수 .ver2`를 이용하여 `주문 내역 생성 함수` 수정하기.

> `주의사항` 구현의 편의를 위하여, 적재데이터의 columns의 순서가 바뀌었음. (dependent한 특성들 + independent한 특성)
>> 아래코드 참고!!

In [94]:
columns=['플랜트', '플랜트명', '매장코드', '매장', '상품코드', '상품명', '차수', '주문일자','운송장번호',
       '운송장출력여부', '주문취소여부', '주문번호', '받는이', '우편번호', '주소1', '주소2', '전화번호1',
       '전화번호2', '배송기재사항', '판매가', '합포여부', '예정수량', '검수수량', '출력일시', '출력자', '몰구분',
       'DAS여부', 'DAS차수', 'DAS지시번호', 'BOXNO', 'CELLNO', '추가박스여부']

random_columns = ['차수','주문일자','운송장번호','운송장출력여부','주문취소여부','주문번호','받는이','우편번호','주소1','주소2','전화번호1','전화번호2',
                  '배송기재사항','판매가', '합포여부', '예정수량', '검수수량', '출력일시', '출력자', '몰구분','DAS여부', 'DAS차수', 'DAS지시번호', 'BOXNO', 'CELLNO', '추가박스여부']
dependent_columns = ['플랜트','상품코드']

In [95]:
Dependent_set = {}
for i in range(len(dependent_list)):                       # 확장성 고려하여 다음과 같이 표현.
    f = np.loadtxt('C:/Users/oryuk/OneDrive/바탕 화면/ffm-logistics-planning/relation '+ str(i+1) + '.txt', delimiter='|',dtype = 'str')
    Dependent_set[dependent_list[i][0]] = f
    
def one_column_random2(name):
    return random.choice(Dependent_set[name])

# result=[]
# for i in one_column_random2('플랜트'):
#     result.append(i)

In [96]:
#샘플데이터 받아서 딕셔너리 자료형으로 변환 특성 종류 : 특성 value
Sample_set = {}
for col in columns:
    f = open('C:/geunskoo-github-blog/ffm-logistics-planning/CODE/Sample/' + col,'r',encoding ='utf-8')     #안호성이 만든 CODE/Sample/         각자의 저장 주소 참고하세요!
    data = f.read()[:-1].split('|')
    Sample_set[col] = data

#input - 특성 종류
#output - 랜덤 특성 value
def one_column_random(name):
    return random.choice(Sample_set[name])

#input - 주문 수, "excel파일 이름"
#output - excel 파일 생성
def random_order_creater(n, name = 'Random_Sampe'):
    result = pd.DataFrame([],columns=columns)
    for _ in range(n):
        new_random_data = []
        for col in dependent_columns:
            for i in one_column_random2(col):
                new_random_data.append(i)
                
        for col in random_columns:
            new_random_data.append(one_column_random(col))
        result.loc[len(result)] = new_random_data
    result.to_excel(name+'.xlsx',encoding='utf-8')
    print('파일이 생성되었습니다.')

#실행
random_order_creater(1000,'주문 내역 생성(특성 연결)')

파일이 생성되었습니다.
